<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/Spaceship-Titanic/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install -Uqq fastcore --upgrade
!pip install -Uqq fastai --upgrade
!pip install kaggle
import pandas as pd
from fastai.vision.all import *
from fastai.tabular.all import *

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
%%capture
!kaggle competitions download -c spaceship-titanic

In [5]:
%%capture
!unzip spaceship-titanic.zip

In [6]:
%%capture
!unzip spaceship-titanic.zip

In [10]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [8]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [17]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

In [43]:
to = TabularPandas(df_train, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['HomePlanet', 'CryoSleep','Destination', 'VIP'],
                   cont_names = ['Age','FoodCourt','ShoppingMall','Spa','VRDeck'],
                   y_names='Transported',
                   splits=splits)

In [19]:
to.xs.iloc[:2]

,HomePlanet,CryoSleep,Destination,VIP,Age_na,FoodCourt_na,ShoppingMall_na,Spa_na,VRDeck_na,Age,FoodCourt,ShoppingMall,Spa,VRDeck
7591,2,1,3,1,1,1,1,1,1,-0.335611,0.220678,-0.280429,4.292261,-0.134744
6291,1,0,3,1,1,1,1,1,1,-0.405522,-0.279370,-0.280429,-0.272316,-0.261202


In [36]:
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(df_test, with_labels=True)

In [21]:
dls.show_batch()

,HomePlanet,CryoSleep,Destination,VIP,Age_na,FoodCourt_na,ShoppingMall_na,Spa_na,VRDeck_na,Age,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Mars,False,TRAPPIST-1e,False,False,False,False,False,False,48.0,-0.000010,877.000001,-0.000008,-0.000010,True
1,Mars,True,TRAPPIST-1e,False,False,False,False,False,False,20.0,-0.000010,-0.000001,-0.000008,-0.000010,True
2,Earth,False,TRAPPIST-1e,False,False,False,False,False,False,5.0,-0.000010,-0.000001,-0.000008,-0.000010,True
3,Mars,#na#,#na#,False,False,False,False,False,False,53.0,-0.000010,2.000001,0.999995,11.000001,False
4,Europa,True,TRAPPIST-1e,False,False,False,False,False,False,15.0,-0.000010,-0.000001,-0.000008,-0.000010,True
5,Earth,False,TRAPPIST-1e,False,False,False,False,False,False,27.0,1108.999995,921.000038,-0.000008,0.999997,False
6,Earth,False,TRAPPIST-1e,False,False,False,False,False,False,24.0,871.000015,16.999995,-0.000008,-0.000010,False
7,Earth,True,TRAPPIST-1e,False,False,False,False,False,False,56.0,-0.000010,-0.000001,-0.000008,-0.000010,False
8,Mars,False,TRAPPIST-1e,False,False,False,False,False,False,28.0,-0.000010,659.000033,607.000009,-0.000010,False
9,Europa,False,55 Cancri e,False,False,False,False,False,False,30.0,3627.999911,289.000000,1736.999952,4987.000338,False


In [37]:
model = tabular_learner(dls, metrics=accuracy)

In [38]:
model.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.550562,0.457372,0.779056,00:01
1,0.486890,0.452464,0.793441,00:01
2,0.471410,0.437473,0.810127,00:01


In [24]:
model.show_results()

,HomePlanet,CryoSleep,Destination,VIP,Age_na,FoodCourt_na,ShoppingMall_na,Spa_na,VRDeck_na,Age,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Transported_pred
0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.852870,-0.279370,-0.280429,-0.272316,-0.261202,1.0,1.0
1,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,0.363495,-0.272571,0.730364,0.275100,0.418839,1.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.055969,-0.279370,0.223311,-0.002776,-0.259433,1.0,0.0
3,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.244450,-0.279370,-0.280429,-0.272316,-0.261202,0.0,1.0
4,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.125879,-0.279370,-0.280429,-0.272316,-0.261202,1.0,0.0
5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.873646,-0.279370,-0.280429,-0.272316,-0.261202,1.0,1.0
6,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.782959,-0.266390,-0.280429,1.119843,0.969769,0.0,0.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.545343,-0.256500,-0.280429,-0.272316,-0.239978,0.0,0.0
8,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,-0.405522,-0.279370,-0.053415,-0.175986,-0.254127,0.0,0.0


In [28]:
row, clas, probs = model.predict(df_test.iloc[0])

In [29]:
row.show()

,HomePlanet,CryoSleep,Destination,VIP,Age_na,FoodCourt_na,ShoppingMall_na,Spa_na,VRDeck_na,Age,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Earth,True,TRAPPIST-1e,False,False,False,False,False,False,27.0,-0.00001,-0.000001,-0.000008,-0.00001,False


In [39]:
preds = model.get_preds(dl=test_dl)
model.validate(dl=test_dl)

(#2) [None,None]

In [68]:
preds[0]

tensor([[0.4666, 0.5334],
        [0.9561, 0.0439],
        [0.0366, 0.9634],
        ...,
        [0.1203, 0.8797],
        [0.3535, 0.6465],
        [0.4287, 0.5713]])

In [62]:
test_preds = np.exp(preds[0]).numpy().T[0]

In [63]:
df_test['Transported']  = preds[0]

In [64]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0.466635
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0.956114
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0.036599
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0.152526
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0.645674


In [65]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0.466635
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0.956114
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0.036599
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0.152526
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0.645674


In [79]:
df_test[0:]['Transported']

0       0.466635
1       0.956114
2       0.036599
3       0.152526
4       0.645674
          ...   
4272    0.460467
4273    0.637018
4274    0.120268
4275    0.353459
4276    0.428664
Name: Transported, Length: 4277, dtype: float32

In [80]:
list_v = []
for row in df_test[0:]['Transported']:
  rr = float(row)
  if rr>0.5:
    list_v.append(True)
  else:
     list_v.append(False)


In [82]:
%%capture
list_v

In [83]:
df_test['Transported']  = list_v

In [84]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,False
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,True
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,False
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,False
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,True


In [85]:
df_test[['PassengerId', 'Transported']].to_csv('submission.csv', index=False)